# Введение

Есть много библиотек для работы с различными параметризациями поворотов. `Scipy.spatial.transform`, `Transforms3d`, `Pytransform3d`, `Numpy-quaternion`, `Blender.mathutils`.

Мы будем использовать `scipy.spatial.transform`. Установите scipy, если еще не установлен `pip install scipy`

In [ ]:
%pip install scipy numpy

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation
from typing import List, Tuple

Класс `scipy.spatial.transform.Rotation` представляет трехмерный поворот. В этом классе вы найдете методы для перевода поворотов из одной параметризации в другую и некоторые другие. Например метод для генерации случайных равномерно распределенных поворотов.

In [ ]:
# в более старых версиях scipy методы `from_matrix`, `as_matrix` называются `from_dcm`, `as_dcm`
r = Rotation.from_matrix(np.eye(3))

print('Identity поворот')
print('в виде матрицы')
print(r.as_matrix())
print('углов Родрига')
print(r.as_rotvec())
print('кватерниона')
print(r.as_quat())
print('углов Эйлера (zyx)')
print(r.as_euler('zyx', degrees=True))

r = Rotation.from_euler('zyx', (0, 90, 0), degrees=True)
print()
print('Поворот вокруг оси y на 90 градусов')
print('в виде матрицы')
print(r.as_matrix())
print('углов Родрига')
print(r.as_rotvec())
print('кватерниона')
print(r.as_quat())
print('углов Эйлера (zxy)')
print(r.as_euler('zxy', degrees=True))
print('углов Эйлера (zyx). Обратите внимание, это Gimbal Lock при таком порядке углов')
print(r.as_euler('zyx', degrees=True))

r = r * r
print()
print('Поворот вокруг оси y на 180 градусов')
print('в виде матрицы')
print(r.as_matrix())
print('углов Родрига')
print(r.as_rotvec())
print('кватерниона')
print(r.as_quat())
print('углов Эйлера (zxy)')
print(r.as_euler('zxy', degrees=True))
print('углов Эйлера (zyx)')
print(r.as_euler('zyx', degrees=True))

Нам пригодятся методы применения трансформаций. Они переписаны чтобы использовать `Rotation` вместо матрицы поворота. Также `transform` теперь может принимать массив точек.

In [ ]:
def transform(points_3d: np.array, r: Rotation, t: np.array):
    return r.apply(points_3d) + t

def compose_transforms(r1: Rotation, t1: np.array, r2: Rotation, t2: np.array):
    return r2 * r1, r2.apply(t1) + t2

def inverse_transform(r: np.array, t: np.array):
    return r.inv(), -r.inv().apply(t)

# Задание 1

Напишите 3 функции, которые возвращают интерполированные между двумя позициями `r1, t1` и `r2, t2` положения точек `points`. При `time == 0` точки толжны трансформироваться `r1, t1`, при `time == 1` – `r2, t2`

- `interpolate_euler` – должна интерполировать поворот интерполируя углы Эйлера (в любом порядке. Например `zyx`)
- `interpolate_lerp` – должна интерполировать поворот линейно интерполируя матрицы поворота. Обратите внимание, тут нельзя использовать класс `Rotation` т.к. интерполированные матрицы не будут матрицами поворота
- `interpolate_slerp` – должна интерполировать поворот используя `Slerp`

In [ ]:
from scipy.spatial.transform import Slerp


def interpolate_euler(points: np.array, r1: Rotation, t1: np.array, r2: Rotation, t2: np.array, time: float) -> np.array:
    pass

def interpolate_lerp(points: np.array, r1: Rotation, t1: np.array, r2: Rotation, t2: np.array, time: float) -> np.array:
    pass

def interpolate_slerp(points: np.array, r1: Rotation, t1: np.array, r2: Rotation, t2: np.array, time: float) -> np.array:
    pass


Тут можно посмотреть как разные интерполяции выглядят в 3d.

In [ ]:
# случайное облако точек, половина точек растянута вдоль оси y, половина вроль оси x (должно выглядеть как крестик)
points = np.random.normal(scale=1, size=(50, 3))
points[points.shape[0] // 2:, 0] *= 2
points[points.shape[0] // 2:, 1] /= 2
points[:points.shape[0] // 2, 1] *= 4


# можно поменять позиции чтобы посмотреть другие примеры
r0, t0 = Rotation.from_euler('zyx', (60, -30, -20), degrees=True), np.array([0, 0, 0])
r1, t1 = Rotation.from_euler('zyx', (-150, 0, 45), degrees=True), np.array([0, 0, 0])

import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook
plt.rcParams['figure.dpi'] = 110 # поменяйте значение чтобы поменять размер графика
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim3d(-10, 10)
ax.set_ylim3d(-10, 10)
ax.set_zlim3d(-10, 10)
colors = np.arange(points.shape[0]) / points.shape[0]
graph = ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=colors, cmap='hsv')

def update(time, interpolation_function):
    points_t = interpolation_function(points, r0, t0, r1, t1, time)
    graph._offsets3d = (points_t[:, 0], points_t[:, 1], points_t[:, 2])
    fig.canvas.draw_idle()

interact(update,
         time=(0.0, 1.0, 0.01),
         interpolation_function=[('euler', interpolate_euler), ('lerp', interpolate_lerp), ('slerp', interpolate_slerp)])

# Задание 2

У вас есть наборы 3D точек `points1`, `points2`. Точек как минимум `3`.

Напишите функцию `detect_transform`, которая по двум наборам 3D точек находит позицию `(r, t)`, которая переводит точки `points1` в точки `points2`. Тоесть `transform(points1, r, t)` близко к `points2`.

Используйте функцию `Rotation.align_vectors(a, b)`. Она находит поворот `r`, такой что `||r.apply(b) - a|| -> min`.

*Обратите внимание. `r.apply(b) - a`, а не `r.apply(a) - b`*

In [ ]:
def detect_transform(points1: np.array, points2: np.array) -> Tuple[Rotation, np.array]:
    pass


def test_detect_transform(r, t, noise_scale):
    points1 = np.array([[1, 0, 1],
                        [-1, 0, 0],
                        [1, 0, 0],
                        [32, 21, 99]])
    points2 = transform(points1, r, t) + np.random.normal(scale=noise_scale, size=points1.shape)
    r_found, t_found = detect_transform(points1, points2)
    points2_transformed = transform(points1, r_found, t_found)
    assert(np.linalg.norm(points2 - points2_transformed) < 1e-3 + 10 * noise_scale)
    
test_detect_transform(Rotation.identity(), np.zeros(3), 0)
test_detect_transform(Rotation.identity(), np.array([92, -2, -1.23456]), 1e-4)
test_detect_transform(Rotation.from_euler('zyx', (0, 0, 90), degrees=True), np.zeros(3), 1e-3)
test_detect_transform(Rotation.from_euler('xyz', (32, 12, -67), degrees=True), np.array([0, -3, -12]), 1e-3)

# Задание 3
RANSAC – алгоритм нахождения параметров модели по измерениям с выбросами.

Если предпологать что в исходных данных не более `max_outliers_proportion` выбросов и для того чтобы построить гипотезу нужна выборка из `sample_size` измерений, то для фиксированного числа `iterations_count` можно посчитать вероятность того, что RANSAC найдет правильные параметры модели (построит хотябы одну гипотезу по инлаерам).

- напишите функцию `calculate_ransac_success_probability`, которая считает вероятность успешной работы RANSAC
- напишите функцию `calculate_ransac_iterations`, которая считает необходимое число итераций чтобы вероятность успешной работы RANSAC была не меньше `target_success_probability`

In [ ]:
def calculate_ransac_success_probability(max_outliers_proportion: float,
                                         sample_size: int,
                                         iterations_count: int) -> float:
    pass


def calculate_ransac_iterations(max_outliers_proportion: float,
                                sample_size: int,
                                target_success_probability: float) -> int:
    pass


assert(np.isclose(calculate_ransac_success_probability(0.5, 1, 1), 0.5))
assert(np.isclose(calculate_ransac_success_probability(0.5, 2, 3), 0.578125))
assert(calculate_ransac_iterations(0.6, 1, 0.2) == 1)
assert(calculate_ransac_iterations(0.6, 7, 0.2) == 137)
assert(np.isclose(calculate_ransac_success_probability(0.5, 2, calculate_ransac_iterations(0.5, 2, 0.9924)),
                  0.9924, 
                  atol=1e-3))

RANSAC сильно помогает в случаях когда в исходных данных имеются выбросы. А в реальных данных такое встречается часто.

RANSAC так же может быть узким местом в смысле производительности.

In [ ]:
target_p = 0.999
print('Хочется чтобы вероятность что RANSAC сойдется была велика. Например %0.1f%%' % (target_p * 100, ))

target_outliers = 0.5
print('Также хочется чтобы RANSAC справлялся при как можно большей долей возможных выбросов. Например %0.1f%%' % 
      (target_outliers * 100, ))

print('')
print('Для алгоритма поиска линии, где гипотеза строится по 2 измерениям, \n'
      'нужно всего %d итераций чтобы получить требуемые параметры\n' %
      calculate_ransac_iterations(target_outliers, 2, target_p))

print('Для алгоритма поиска 3d позиции по 3d-3d соответствиям, '
      'где гипотеза строится по 3 измерениям, \n'
      'нужно уже %d итераций чтобы получить требуемые параметры\n' %
      calculate_ransac_iterations(target_outliers, 3, target_p))

print('Для алгоритма поиска 3d позиции камеры по 3d-2d соответствиям, '
      'где гипотеза строится по 4 измерениям, \n'
      'нужно %d итераций чтобы получить требуемые параметры\n' %
      calculate_ransac_iterations(target_outliers, 4, target_p))

target_outliers_2 = 0.75
print('А если считать что выбросов может быть %0.1f%%, нужно %d итераций\n' %
      (target_outliers_2 * 100, calculate_ransac_iterations(target_outliers_2, 4, target_p)))

print('При этом если для построения гипотезы нужна бОльшая выборка, например 7, нужно %d итераций\n' %
      calculate_ransac_iterations(target_outliers_2, 7, target_p))

# Задание 4

У вас есть все необходимые методы для того чтобы написать устойчивый к выбросам метод `detect_transform_ransac`.

Используйте `detect_transform` на случайных выборках из `points1`, `points2` чтобы строить гипотезу `r, t`. Используйте `find_inliers` чтобы находить инлаеров для построенной гипотезы.

`detect_transform_ransac` должен строить ровно `iterations_count` гипотез по выборкам размера `sample_size` и возвращать лучшую найденную гипотезу вместе с маской инлаеров для нее: `r_best, t_best, inliers_mask_best = detect_transform_ransac(...)`

In [ ]:
def find_inliers(points1: np.array, points2: np.array, r: Rotation, t: np.array, eps=1e-2) -> np.array:
    return np.linalg.norm(transform(points1, r, t) - points2, axis=1) < eps

In [ ]:
def detect_transform_ransac(points1: np.array,
                            points2: np.array,
                            sample_size: int, iterations_count: int) -> Tuple[Rotation, np.array, np.array]:
    pass


def test_detect_transform_ransac(r, t, inliers_ratio, sample_size, iterations_count, noise_scale=1e-3):
    points_count = 1000
    inliers_count = int(points_count * inliers_ratio)
    outliers_count = points_count - inliers_count
    
    expected_succ_chance = calculate_ransac_success_probability(1 - inliers_ratio, sample_size, iterations_count)
    print('RANSAC with %d inliers and %d outliers, sample size %d and %d iterations '
          'has %2.2f%% probability to succeed' % 
          (inliers_count, outliers_count, sample_size, iterations_count, expected_succ_chance * 100))
    
    points1_inliers = np.random.normal(size=(inliers_count, 3))
    points2_inliers = transform(points1_inliers, r, t) + np.random.normal(scale=noise_scale, size=points1_inliers.shape)
    points1_outliers = np.random.normal(size=(outliers_count, 3))
    points2_outliers = np.random.normal(size=(outliers_count, 3))
    
    shuffle_idxs = np.arange(points_count)
    np.random.shuffle(shuffle_idxs)
    points1 = np.concatenate([points1_inliers, points1_outliers])[shuffle_idxs]
    points2 = np.concatenate([points2_inliers, points2_outliers])[shuffle_idxs]
    
    print('\trunning detect_transform on inliers + outliers')
    r_found, t_found = detect_transform(points1, points2)
    error_norm = np.linalg.norm(points2_inliers - transform(points1_inliers, r_found, t_found))
    print('\tfound solution error norm is %0.3f' % error_norm)
    
    print('\trunning RANSAC')
    r_found, t_found, inliers_mask = detect_transform_ransac(points1, points2, sample_size, iterations_count)
    
    inliers_found = np.count_nonzero(inliers_mask)
    print('\tsolution with %d inliers found' % inliers_found)
    if inliers_found < inliers_count:
        print('\tRANSAC failed!')
        return
    
    error_norm = np.linalg.norm(points2_inliers - transform(points1_inliers, r_found, t_found))
    print('\tfound solution error norm is %0.3f' % error_norm)
    assert(error_norm < 1e-1 + 100 * noise_scale)
    
    print('\trunning detect_transform on inliers')
    r_found, t_found = detect_transform(points1[np.where(inliers_mask)], points2[np.where(inliers_mask)])
    error_norm = np.linalg.norm(points2_inliers - transform(points1_inliers, r_found, t_found))
    print('\tfound solution error norm is %0.3f' % error_norm)
    assert(error_norm < 1e-2 + 100 * noise_scale)



print('This one should work OK:')
test_detect_transform_ransac(Rotation.from_euler('xyz', (1, -2, -0.32)), np.array([1, -3.4, 9]), 0.5, 4, 100)

print('')
print('This one may not work OK:')
test_detect_transform_ransac(Rotation.from_euler('xyz', (1, -2, -0.32)), np.array([1, -3.4, 9]), 0.5, 10, 100)

# Задание 5*

Мы хотим оценить качество работы алгоритма отслеживания камеры. Результатом работы алгоритма отслеживания камеры является путь камеры в некоторой сцене – список пар `r, t` позиций камеры в каждом кадре.

`track1` – это *правильный* путь камеры (например полученный измерением реального положения камеры во время съемки, либо полученный с помощью *эталонного* алгоритма отслеживания камеры)

`track2` – это посчитанный алгоритмом путь камеры

Напишите функцию `compare_tracks`, которая возвращает число `float >= 0` – *некую* ошибку(разницу) между `track1`, `track2`. Если два пути одинаковые то `compare_tracks` должна возвращать 0. 

Гарантируется что
- `len(track1) == len(track2)`
- `len(track1) >= 4`
- оба пути имеют ненулевую длину

Обратите внимание что
- `track1`, `track2` могут иметь разный масштаб (т.к. алгоритм отслеживания камеры не может определить масштаб сцены)
- `track1`, `track2` могут начинаються из разной позиции
- разный масштаб не должен влиять на значение `compare_tracks`

In [ ]:
def compare_tracks(track1: List[Tuple[Rotation, np.array]],
                   track2: List[Tuple[Rotation, np.array]]) -> float:
    pass


track_len = 10
track1 = [(Rotation.from_euler('zyx', (10, -2 * i, i), degrees=True), 
           np.array([i / track_len * 10 - 5, 2, (i / track_len) ** 2]))
          for i in range(track_len)]
track2 = track1[:]

same_track_error = compare_tracks(track1, track2)
assert(np.isclose(same_track_error, 0))

track2 = [(r, t * 3) for r, t in track2]
same_track_different_scale_error = compare_tracks(track1, track2)
assert(np.isclose(same_track_different_scale_error, 0, atol=1e-3))

offset_r = Rotation.from_euler('zyx', (0, 1, 0.2))
offset_t = np.ones(3)
track2 = [compose_transforms(r, t, offset_r, offset_t) for r, t in track2]
same_track_different_scale_and_pos_error = compare_tracks(track1, track2)
assert(np.isclose(same_track_different_scale_and_pos_error, 0, atol=1e-3))

track2[track_len // 2] = (track2[track_len // 2][0], track2[track_len // 2][1] + np.array([1, 0, 0.2]))
one_frame_wrong_t_error = compare_tracks(track1, track2)
assert(one_frame_wrong_t_error > 0 and not np.isclose(one_frame_wrong_t_error, 0))

track2[track_len // 3] = compose_transforms(offset_r, np.zeros(3),
                                            track2[track_len // 3][0], track2[track_len // 3][1])
two_frame_wrong_r_t_error = compare_tracks(track1, track2)
assert(two_frame_wrong_r_t_error > one_frame_wrong_t_error)